In [ ]:

# coding: utf-8

# In[1]:


import skater
import sklearn
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn import preprocessing
import matplotlib as plt
from sklearn.metrics import roc_curve, auc
import seaborn as sns


# In[2]:


link = '/home/abhi/Downloads/Kaggle/HomeCreditDefaultRisk/application_train.csv'


# In[3]:


data = pd.read_csv(link)
data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].replace(365243.000000,np.nan)
data['DAYS_BIRTH']= data['DAYS_BIRTH'] / -365
data['DAYS_ID_PUBLISH'][data['DAYS_BIRTH'] > 45.0] = data['DAYS_ID_PUBLISH'] - 17.5
data['DAYS_ID_PUBLISH']= data['DAYS_ID_PUBLISH'] / -365
data['DAYS_REGISTRATION'] = data['DAYS_REGISTRATION'] /-365


# In[4]:


missing_col= data.columns[data.isnull().any()]


# In[5]:


missing_col = list(missing_col)


# In[6]:


missing_col_df = data[list(missing_col)]


# In[7]:


non_missing_df = data[list(set(data.columns)-set(missing_col))]


# In[8]:


non_missing_obj_df = non_missing_df.select_dtypes('object')


# In[9]:


non_missing_obj_dummy_df = pd.get_dummies(non_missing_obj_df)  # add--------------------------------


# In[10]:


non_missing_numerical_df = non_missing_df[list(set(non_missing_df.columns)-set(non_missing_obj_df.columns))] # add-


# In[11]:


df_obj = missing_col_df.select_dtypes('object')


# In[12]:


df_obj_dummy = pd.get_dummies(df_obj) # add -----------------------------------------


# In[13]:


data_obj_col = set(df_obj.columns)


# In[14]:


data_num_col = set(missing_col_df.columns) - data_obj_col


# In[15]:


data_num_col = list(data_num_col)


# In[16]:


df_numerical = missing_col_df[data_num_col]


# In[17]:


df_numerical_notnan = df_numerical.fillna(0) # add-----------------------------------------


# In[18]:


df_numerical_nan = df_numerical - df_numerical


# In[19]:


df_numerical_nan = df_numerical_nan.replace(np.nan, 1) # add-------------------------------------


# In[20]:


df_numerical_nan_col = map(str.lower,list(df_numerical_nan.columns))


# In[21]:


df_numerical_nan.columns = df_numerical_nan_col


# In[22]:


df_numerical_final = pd.concat([non_missing_obj_dummy_df, non_missing_numerical_df,
                                df_obj_dummy, df_numerical_notnan,df_numerical_nan], axis=1) 


# In[23]:





# In[24]:


# DATA SPLIT
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df_numerical_final, df_numerical_final['TARGET']):
    train_set = df_numerical_final.loc[train_index]
    test_set = df_numerical_final.loc[test_index]

X_train = train_set.drop(['TARGET'],axis=1)
y_train = train_set['TARGET']
X_test = test_set.drop(['TARGET'],axis=1)

y_test = test_set['TARGET']



X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
y_test = y_test.values
y_train = y_train.values

# In[28]:
print('RND CLASSIFIER.....................')


rnd_clf = RandomForestClassifier(n_estimators=15, min_samples_leaf=2,
                                 min_samples_split=2, class_weight='balanced',random_state=0)
rnd_clf.fit(X_train, y_train)
preds_rnd = rnd_clf.predict(X_test)

print(confusion_matrix(y_test, preds_rnd))


# In[29]:




# In[30]:


from sklearn.metrics import roc_curve, auc, roc_auc_score
print(roc_auc_score(y_test, rnd_clf.predict_proba(X_test)[:,1]))


# In[31]:


from sklearn.metrics import roc_curve, auc, roc_auc_score
print(roc_auc_score(y_test, rnd_clf.predict_proba(X_test)[:,1]))


# In[32]:
print('GAUSSIAN CLASSIFIER.....................')


gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
preds_gnb = gnb_clf.predict(X_test)

print(confusion_matrix(y_test, preds_gnb))


# In[33]:


from sklearn.metrics import roc_curve, auc, roc_auc_score
print(roc_auc_score(y_test, gnb_clf.predict_proba(X_test)[:,1]))


# In[34]:

print('BAGGING CLASSIFIER.....................')


# In[39]:


bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=10,
max_samples=0.8 ,max_features=1.0,bootstrap_features=True,
bootstrap=False,
n_jobs=-1
)


# In[ ]:


bag_clf.fit(X_train, y_train)
preds_bag = bag_clf.predict(X_test)

print(confusion_matrix(y_test, preds_bag))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds_bag))


# In[ ]:


from sklearn.metrics import roc_curve, auc, roc_auc_score
print(roc_auc_score(y_test, bag_clf.predict_proba(X_test)[:,1]))


print('SVM CLASSIFIER.....................')



polynomial_svm_clf =  SVC(C=20, probability=True, degree=2)
polynomial_svm_clf.fit(X_train,y_train)
preds_svm = polynomial_svm_clf.predict(X_test)

print(confusion_matrix(y_test, preds_svm))
print(classification_report(y_test, preds_svm))


# In[ ]:


from sklearn.metrics import roc_curve, auc, roc_auc_score
print(roc_auc_score(y_test,polynomial_svm_clf.predict_proba(X_test)[:,1]))

print('XGBOOST CLASSIFIER.....................')

# In[ ]:



print('LDA CLASSIFIER.....................')


# In[ ]:


lda_clf = LinearDiscriminantAnalysis(n_components = 4)
lda_clf.fit(X_train, y_train)
preds_lda = lda_clf.predict(X_test)

print(confusion_matrix(y_test, preds_lda))
print(classification_report(y_test, preds_lda))


# In[ ]:


from sklearn.metrics import roc_curve, auc, roc_auc_score
print(roc_auc_score(y_test,lda_clf.predict_proba(X_test)[:,1]))


# In[ ]:

print('.................FINAL VOTING CLASSIFIER.....................')







/home/abhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


RND CLASSIFIER.....................
[[56234   304]
 [ 4814   151]]
0.6702396131938747
0.6702396131938747
GAUSSIAN CLASSIFIER.....................
[[ 1881 54657]
 [   66  4899]]
0.5114906061629111
BAGGING CLASSIFIER.....................
[[56420   118]
 [ 4917    48]]
             precision    recall  f1-score   support

          0       0.92      1.00      0.96     56538
          1       0.29      0.01      0.02      4965

avg / total       0.87      0.92      0.88     61503

0.6338388939065018
SVM CLASSIFIER.....................


In [ ]:

voting_clf = VotingClassifier(
estimators=[('bag_clf', bag_clf), ('polynomial_svm_clf',polynomial_svm_clf),
            ('rnd_clf', rnd_clf), ('xgb_clf', xgb_clf), ('gnb_clf', gnb_clf), ('lda_clf', lda_clf)],
    weights=[3,3,4,3,4,4],
voting='soft'
)
voting_clf.fit(X_train,y_train)


# In[ ]:


preds_vtg = voting_clf.predict(X_test)
print(confusion_matrix(y_test, preds_vtg))
print(classification_report(y_test, preds_vtg))


# In[ ]:


from sklearn.metrics import roc_curve, auc, roc_auc_score
print(roc_auc_score(y_test,voting_clf.predict_proba(X_test)[:,1]))